In [2]:
import pandas as pd
import psycopg2
from rbbm_src.dc_src.DCRepair import (eq_op, 
 non_symetric_op,
#  dc_tuple_violation_template_targeted_t1,
#  dc_tuple_violation_template_targeted_t2
)
import re
from rbbm_src.dc_src.src.classes import parse_rule_to_where_clause
from string import Template

In [3]:
conn=psycopg2.connect('dbname=cr user=postgres')
dc_tuple_violation_template_targeted_t1=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")
dc_tuple_violation_template_targeted_t2=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")

In [4]:
cols_q = f"select * from flights_new limit 1"
df_row = pd.read_sql(cols_q, conn)

In [5]:
cols=list(df_row)

In [6]:
def construct_query_for_violation(role, dc_text, target_table, targeted, cols): 
    predicates = dc_text.split('&')
#     clause = parse_rule_to_where_clause(dc_text)
    constants=[]
    # print(f"t_interest:{t_interest}")
    need_tid=True 
    # if the constraint only has equals, we need to add an artificial
    # key (_tid_) to differentiate tuples in violation with the tuple it
    # self
    for pred in predicates[2:]:
        if(not eq_op.search(pred)):
            need_tid=False
        attr = re.search(r't[1|2]\.([-\w]+)', pred).group(1).lower()
        # print(attr)
#         constants.append(f'{role}.{attr}=\'{t_interest[attr]}\'')
    constants_clause = ' AND '.join(constants)
    # print(f"dc_text:{dc_text}")
    t1_cols=', '.join([f't1.{x} as t1_{x}' for x in cols])
    t2_cols=', '.join([f't2.{x} as t2_{x}' for x in cols])
    
    if(role=='t1'):
        template=dc_tuple_violation_template_targeted_t1
    else:
        template=dc_tuple_violation_template_targeted_t2
    if(targeted):
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text),
                                           tuple_desc=constants_clause)

    else:
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text))

        r_q+=f" AND t1._tid_!=t2._tid_"
#     print(r_q)

    return r_q

def find_tuples_in_violation(conn, dc_text, target_table, cols, targeted=False):
    if(non_symetric_op.search(dc_text)):
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted)
        q2 = construct_query_for_violation('t2', dc_text, target_table, targeted)
        res = {'t1': pd.read_sql(q1, conn).to_dict('records'), 
        't2': pd.read_sql(q2, conn).to_dict('records')}
    else:
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted, cols)
        print(q1)
        res = {'t1':pd.read_sql(q1, conn).to_dict('records')}
        # print(res)
    return res 

In [7]:
dc_file='/home/opc/author/RBBM/rbbm_src/muse/data/mas/flights_dcfinder_rules.txt'
table_name='flights_new'
res=[]
rule_texts=[]
try:
    with open(dc_file, "r") as file:
        for line in file:
            rule_texts.append(line.strip())
            # rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'adult', 't1')]
#             rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'flights_new', 't1')]
#             res.extend(rules_from_line)
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")


In [8]:
print(len(rule_texts))

83


In [9]:
dirty_tuples = set()

In [10]:
rules_with_stats={r:{} for r in rule_texts}

In [11]:
for r in rule_texts:
    res = find_tuples_in_violation(conn, r, table_name, cols)
    print(f"rule: {r}")
    for k in res:
        for pair in res[k]:
#             if(pair['t1_is_dirty']!=pair['t2_is_dirty']):
            dirty_tuples.add(pair['t1__tid_'])
            dirty_tuples.add(pair['t2__tid_'])
        len_cc = len([x for x in res[k] if (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        len_non_cc = len([x for x in res[k] if not (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        len_dd = len([x for x in res[k] if (x['t1_is_dirty']==True and x['t2_is_dirty']==True)])
        len_dc = len([x for x in res[k] if ((x['t1_is_dirty']==True and x['t2_is_dirty']==False) or \
                    (x['t1_is_dirty']==False and x['t2_is_dirty']==True))])
        rules_with_stats[r]['len_cc']=len_cc
        rules_with_stats[r]['len_dd']=len_dd
        rules_with_stats[r]['len_dc']=len_dc
        print(f"number of ccs:{len_cc}, number of non ccs: {len_non_cc}, number of dds: {len_dd}, number of dcs:{len_dc}")
#     break
    print('\n')

SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.fl_date as t2_fl_date, t2.op_unique_carrier as t2_op_unique_carrier, t2.op_carrier_fl_num as t2_op_carrier_fl_num, t2.origin_airport_id as t2_origin_airport_id,

rule: t1&t2&EQ(t1.ORIGIN_CITY_NAME,t2.ORIGIN_CITY_NAME)&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)
number of ccs:1532, number of non ccs: 22, number of dds: 0, number of dcs:22


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_

rule: t1&t2&EQ(t1.DEST_AIRPORT_ID,t2.DEST_AIRPORT_ID)&EQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)
number of ccs:2864, number of non ccs: 88, number of dds: 0, number of dcs:88


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_w

rule: t1&t2&EQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&EQ(t1.FL_DATE,t2.FL_DATE)
number of ccs:1848, number of non ccs: 80, number of dds: 0, number of dcs:80


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.

rule: t1&t2&EQ(t1.DEP_TIME,t2.DEP_TIME)
number of ccs:948, number of non ccs: 30, number of dds: 0, number of dcs:30


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.fl_date as t2_fl_date, t2.op_unique_carri

...

rule: t1&t2&EQ(t1.DISTANCE,t2.DISTANCE)
number of ccs:1456, number of non ccs: 70, number of dds: 0, number of dcs:70


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.fl_date as t2_fl_date, t2.op_unique_carr

rule: t1&t2&EQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)&EQ(t1.ORIGIN_STATE_ABR,t2.ORIGIN_STATE_ABR)
number of ccs:8260, number of non ccs: 352, number of dds: 4, number of dcs:348


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_

rule: t1&t2&EQ(t1.ORIGIN_CITY_NAME,t2.ORIGIN_CITY_NAME)&IQ(t1.ORIGIN_AIRPORT_ID,t2.ORIGIN_AIRPORT_ID)
number of ccs:2576, number of non ccs: 286, number of dds: 0, number of dcs:286


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_wee

rule: t1&t2&IQ(t1.DEST_STATE_NM,t2.DEST_STATE_NM)&IQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)&IQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&IQ(t1.OP_UNIQUE_CARRIER,t2.OP_UNIQUE_CARRIER)
number of ccs:651924, number of non ccs: 26506, number of dds: 240, number of dcs:26266


SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance 

rule: t1&t2&EQ(t1.ORIGIN,t2.ORIGIN)&IQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)
number of ccs:0, number of non ccs: 0, number of dds: 0, number of dcs:0




In [12]:
dirty_tuples

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [13]:
len(dirty_tuples)

1000

In [14]:
rules_with_stats

{'t1&t2&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)&EQ(t1.ORIGIN_AIRPORT_ID,t2.ORIGIN_AIRPORT_ID)': {'len_cc': 1398,
  'len_dd': 0,
  'len_dc': 106},
 't1&t2&IQ(t1.DEST_CITY_NAME,t2.DEST_CITY_NAME)&EQ(t1.DEST_AIRPORT_ID,t2.DEST_AIRPORT_ID)': {'len_cc': 0,
  'len_dd': 0,
  'len_dc': 0},
 't1&t2&IQ(t1.DEST_AIRPORT_ID,t2.DEST_AIRPORT_ID)&EQ(t1.DEST_CITY_NAME,t2.DEST_CITY_NAME)': {'len_cc': 2304,
  'len_dd': 0,
  'len_dc': 84},
 't1&t2&EQ(t1.CRS_DEP_TIME,t2.CRS_DEP_TIME)': {'len_cc': 2894,
  'len_dd': 2,
  'len_dc': 106},
 't1&t2&EQ(t1.ORIGIN_CITY_NAME,t2.ORIGIN_CITY_NAME)&EQ(t1.DEST_CITY_NAME,t2.DEST_CITY_NAME)': {'len_cc': 524,
  'len_dd': 0,
  'len_dc': 8},
 't1&t2&EQ(t1.ORIGIN_CITY_NAME,t2.ORIGIN_CITY_NAME)&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)': {'len_cc': 1532,
  'len_dd': 0,
  'len_dc': 22},
 't1&t2&EQ(t1.DEST_CITY_NAME,t2.DEST_CITY_NAME)&EQ(t1.ORIGIN_AIRPORT_ID,t2.ORIGIN_AIRPORT_ID)': {'len_cc': 478,
  'len_dd': 0,
  'len_dc': 26},
 't1&t2&EQ(t1.ORIGIN_STATE_ABR,t2.ORIGIN_STATE_ABR)&

In [15]:
sorted(rules_with_stats, key=lambda k: rules_with_stats[k]['len_cc'], reverse=True)

['t1&t2&IQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)&IQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&IQ(t1.OP_UNIQUE_CARRIER,t2.OP_UNIQUE_CARRIER)&IQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)',
 't1&t2&IQ(t1.DEST_STATE_NM,t2.DEST_STATE_NM)&IQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)&IQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&IQ(t1.OP_UNIQUE_CARRIER,t2.OP_UNIQUE_CARRIER)',
 't1&t2&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)&EQ(t1.OP_UNIQUE_CARRIER,t2.OP_UNIQUE_CARRIER)',
 't1&t2&EQ(t1.DEST_STATE_NM,t2.DEST_STATE_NM)&EQ(t1.OP_UNIQUE_CARRIER,t2.OP_UNIQUE_CARRIER)',
 't1&t2&EQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&EQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)',
 't1&t2&EQ(t1.DAY_OF_WEEK,t2.DAY_OF_WEEK)&EQ(t1.ORIGIN_STATE_ABR,t2.ORIGIN_STATE_ABR)',
 't1&t2&EQ(t1.ORIGIN_STATE_ABR,t2.ORIGIN_STATE_ABR)&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)',
 't1&t2&EQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&EQ(t1.DEST_STATE_NM,t2.DEST_STATE_NM)',
 't1&t2&EQ(t1.ORIGIN_STATE_NM,t2.ORIGIN_STATE_NM)&EQ(t1.DEST_STATE_ABR,t2.DEST_STATE_ABR)',
 't1&t2&EQ(t1.ORIG

In [16]:
len(list({k:v for k,v in rules_with_stats.items() if v['len_cc']==0}))

25